# カスタムコンテナハンズオン　part2


# Part 2: カスタムコンテナを使って Amazon SageMakerで学習と推論をする


## 環境セットアップ

S3バケットとロールを準備します。

In [ ]:
# S3 prefix
prefix = 'LAB-scikit-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## SageMakerのセッションを取得

SageMakerを操作するためのセッションを取得します。

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## 学習データをS3にアップロード

SageMaker の学習時に利用するデータは、S3 に置く必要があります。ローカルの data ディレクトリにあるデータをS3にアップロードします。
デフォルトでは SageMaker は sagemaker-{region}-{your aws account number} というバケットを使用します。当該バケットがない場合には、自動で新しく作成します。upload_data() メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することも可能です。

In [ ]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

print(data_location)

##  モデルの学習

SageMakerではアルゴリズムの学習する  `Estimator` を作成しまｓ。この  `Estimator` に学習に必要な定義をして `fit` を実行すると学習コンテナが起動して学習を開始します。

Estimaterの引数：
*  __container name__  ECRにあるコンテナを指定します。ハンズオンで作成してコンテナを使います。
*  __role__    Amazon SageMaker がユーザーに代わってタスクを実行するロール.
*  __instance count__   モデルのトレーニングに使用される ML コンピューティングインスタンスの数.
*  __instance type__   モデルのトレーニングに使用される ML コンピューティングインスタンスのタイプ.
*  __output path__    トレーニング結果を保存する S3 へのパス
*  __session__    トレーニングジョブで使用されるその他の AWS サービスとのやり取りを管理するセッションオブジェクト


### VPC の設定
学習コンテナをプライベートVPC内で実行するには、 Estimatorのパラメーターにサブネットとセキュリティグループを指定します。

例

 * subnets=['subnet-002b8823c9926b248','subnet-0bb14b8511a1bfcb3']
 * security_group_ids=['sg-0c88910b00bc4c4b2'] 

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)
print (region)
print (image)

In [ ]:
tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess,
                       subnets=['subnet-012fb80f85fe58d14','subnet-0c1799f3fc936e0b2'],  ## 要更新 ##
                       security_group_ids=['sg-0f6a1f0a4e7a2c01d']                       ## 要更新 ##   
                               )

In [ ]:
tree.fit(data_location)

## モデルのホスティング

### 推論エンドポイントのVPC設定
ホスティング用のプライベートVPCを設定するには、VpcConfigパラメーターにサブネットとセキュリティグループを指定します。 サブネットはAZの異なる複数が必要です。

VPC configの設定例

  vpc_config_override=
          {'Subnets':['subnet-0931f9262f9680ba4','subnet-019763bdd150fb74e'], 
           'SecurityGroupIds':['sg-0a0612496ec95007a'] }

### 推論エンドポイントのデプロイ

推論を行うために学習したモデルをデプロイします。deploy() メソッドでは、デプロイ先エンドポイントのインスタンス数、インスタンスタイプを指定します。こちらもインスタンスタイプを local にすることで，このインスタンス内にエンドポイントを作成します。

In [ ]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer,
                        vpc_config_override={'Subnets':['subnet-012fb80f85fe58d14','subnet-0c1799f3fc936e0b2'], ## 要更新 ##
                                                         'SecurityGroupIds':['sg-0f6a1f0a4e7a2c01d'] }) ## 要更新 ##

### 推論の実行

In [ ]:
shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

In [ ]:
# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)

In [ ]:
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

#### 予測モデルに推論APIでリクエスト

In [ ]:
print(predictor.predict(test_data.values).decode('utf-8'))

### Cleanup
エンドポイントのデプロイ中はコストが常時発生します。　テストが終わったらエンドポイントを削除してコストを抑えましょう

In [ ]:
sess.delete_endpoint(predictor.endpoint)

## Batch Transform によるジョブ実行
入力データをS3に置き、バッチジョブで推論を実行し、その結果をS3に出力します。

### Transform ジョブの作成
`Transformer` を作成します。

*  __instance count__ 推論のインスタンス数
*  __instance type__ 推論のインスタンスタイプ
*  __output path__　推論結果のS3の出力先

In [ ]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv'
                              )


* The __data_location__ 入力データのS3の場所
* The __content_type__ 入力データのMIMEタイプ
* The __split_type__ 入力データのデリミタ
* The __input_filter__ 　入力データの中で参照する最初のカラムID。その前のカラムを無視します。

In [ ]:
transformer.transform(data_location, content_type='text/csv', split_type='Line', input_filter='$[1:]')
transformer.wait()

参照[CreateTransformJob API](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTransformJob.html)

### 結果出力の確認
S3に出力されたデータを確認します。

In [ ]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/iris.csv.out".format(transform_output_folder), '/tmp/iris.csv.out')
with open('/tmp/iris.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))